In [41]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import re
import time
tf.__version__

'1.13.0-rc2'

In [2]:
vocab_lines = open(r'InsuranceQnA-master\vocabulary.txt', encoding='utf-8', errors="ignore").read().split('\n')
question_lines = open(r'InsuranceQnA-master\InsuranceQAquestionanslabelraw.encoded', encoding='utf-8', errors="ignore").read().split('\n')
answer_lines = open(r'InsuranceQnA-master\InsuranceQAlabel2answerraw.encoded', encoding='utf-8', errors="ignore").read().split('\n')

In [3]:
id2line = {}
for line in vocab_lines:
    _line = line.split('\t')
    if len(_line) == 2:
        id2line[_line[0]] = _line[1]

In [4]:
convs, ansid = [], []
for line in question_lines[:-1]:
    _line = line.split('\t')
    ansid.append(_line[2].split(' '))
    convs.append(_line[1])

In [5]:
question_lines[0].split('\t')

['medicare-insurance',
 'idx_1285 idx_1010 idx_467 idx_47610 idx_18488 idx_65760',
 '16696']

In [6]:
convs1 = [ ]
for line in answer_lines[:-1]:
    _line = line.split('\t')
    convs1.append(_line[1])

In [7]:
print(convs[:2])
print(ansid[:2])
print(convs1[:2])

['idx_1285 idx_1010 idx_467 idx_47610 idx_18488 idx_65760', 'idx_3815 idx_604 idx_605 idx_891 idx_136 idx_5293 idx_65761']
[['16696'], ['10277']]
['idx_1 idx_2 idx_3 idx_4 idx_5 idx_6 idx_7 idx_8 idx_9 idx_10 idx_11 idx_12 idx_13 idx_14 idx_3 idx_12 idx_15 idx_16 idx_17 idx_8 idx_18 idx_19 idx_20 idx_21 idx_3 idx_12 idx_14 idx_22 idx_20 idx_23 idx_24 idx_25 idx_26 idx_27 idx_28 idx_29 idx_8 idx_30 idx_19 idx_11 idx_4 idx_31 idx_32 idx_22 idx_33 idx_34 idx_35 idx_36 idx_37 idx_30 idx_38 idx_39 idx_11 idx_40 idx_41 idx_42 idx_43 idx_44 idx_22 idx_45 idx_46 idx_11 idx_47 idx_48 idx_49 idx_18 idx_50 idx_20 idx_44 idx_22 idx_51 idx_14 idx_52 idx_53 idx_22 idx_40 idx_21 idx_3 idx_54 idx_46 idx_11 idx_55 idx_56 idx_57 idx_58 idx_59 idx_60 idx_61 idx_62 idx_8 idx_50 idx_11 idx_45 idx_63 idx_64 idx_3 idx_65 idx_66 idx_3 idx_67 idx_68 idx_69 idx_70 idx_14 idx_3 idx_71 idx_72 idx_73 idx_21 idx_74 idx_5 idx_75 idx_76 idx_12 idx_8 idx_77 idx_78 idx_15 idx_79 idx_49 idx_18 idx_19 idx_11 idx_54 idx_4

In [8]:
count=0
questions, answers = [], []
for a in range(len(ansid)):
      for b in range(len(ansid[a])):
#             print(count)
#             print(a,b)
            questions.append(convs[a])
            count+=1

In [9]:
for a in range(len(ansid)):
      for b in range(len(ansid[a])):
            answers.append(convs1[int(ansid[a][b])-1])

In [10]:
### sample code for above task in my version
# ques=[]
# for i in questions:
#     a=[]
#     que_tokens=i.split(' ')
#     for token in que_tokens:
#         word=id2line[token.strip()]
#         a.append(word)
#     ques.append(' '.join(a))

In [11]:
ques, ans  =[], []
m=0
while m<len(questions):
       i=0
       a=[]
       while i < (len(questions[m].split(' '))):
            a.append(id2line[questions[m].split(' ')[i]])
            i=i+1
       ques.append(' '.join(a))
       m=m+1

In [12]:
n=0
while n<len(answers):  
        j=0
        b=[]
        while j < (len(answers[n].split(' '))):
            b.append(id2line[answers[n].split(' ')[j]])
            j=j+1
        ans.append(' '.join(b))
        n=n+1     

In [13]:
def clean_text(text):
        """Cleaning the text by replacing the abbreviated words with their proper full replacement, and converting all the characters to lower case"""

        text = text.lower()

        text = re.sub(r"i'm", "i am", text)
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"she's", "she is", text)
        text = re.sub(r"it's", "it is", text)
        text = re.sub(r"that's", "that is", text)
        text = re.sub(r"what's", "that is", text)
        text = re.sub(r"where's", "where is", text)
        text = re.sub(r"how's", "how is", text)
        text = re.sub(r"\'ll", " will", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"can't", "cannot", text)
        text = re.sub(r"n't", " not", text)
        text = re.sub(r"n'", "ng", text)
        text = re.sub(r"'bout", "about", text)
        text = re.sub(r"'til", "until", text)
        text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,']", "", text)

        return text

# Applying the 'clean_text()' function on the set of Questions and Answers
clean_questions = []
for question in ques:
    clean_questions.append(clean_text(question))

clean_answers = []    
for answer in ans:
    clean_answers.append(clean_text(answer))

In [14]:
min_line_length, max_line_length = 2, 100
short_questions_temp, short_answers_temp = [], []

i = 0
for question in clean_questions:
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])
    i += 1

In [15]:
short_questions, short_answers = [], []

i = 0
for answer in short_answers_temp:
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
    i += 1

In [16]:
len(short_questions)

19108

In [17]:
len(short_answers)

19108

In [18]:
def pad_sentence_batch(sentence_batch, vocab_to_int):
    """Including <PAD> token in sentence to make all batches of same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [19]:
vocab = {}
for question in short_questions:
    for word in question.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

for answer in short_answers:
    for word in answer.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [20]:
threshold = 1
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1

In [21]:
print("Size of total vocab:", len(vocab))

Size of total vocab: 18983


In [22]:
questions_vocab_to_int = {}
word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        questions_vocab_to_int[word] = word_num
        word_num += 1

answers_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        answers_vocab_to_int[word] = word_num
        word_num += 1

In [23]:
codes = ['<PAD>','<EOS>','<UNK>','<GO>']

for code in codes:
    questions_vocab_to_int[code] = len(questions_vocab_to_int)+1

for code in codes:
    answers_vocab_to_int[code] = len(answers_vocab_to_int)+1

In [24]:
questions_int = []
for question in short_questions:
    ints = []
    for word in question.split():
        if word not in questions_vocab_to_int:
            ints.append(questions_vocab_to_int['<UNK>'])
        else:
            ints.append(questions_vocab_to_int[word])
    questions_int.append(ints)

In [25]:
answers_int = []
for answer in short_answers:
    ints = []
    for word in answer.split():
        if word not in answers_vocab_to_int:
            ints.append(answers_vocab_to_int['<UNK>'])
        else:
            ints.append(answers_vocab_to_int[word])
    answers_int.append(ints)

In [26]:
word_count = 0
unk_count = 0

for question in questions_int:
    for word in question:
        if word == questions_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1

for answer in answers_int:
    for word in answer:
        if word == answers_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1

unk_ratio = round(unk_count/word_count,4)*100

In [27]:
sorted_questions = []
short_questions1 = []
sorted_answers = []
short_answers1= []

for length in range(1, max_line_length+1):
    for i in enumerate(questions_int):
        if len(i[1]) == length:
            sorted_questions.append(questions_int[i[0]])
            short_questions1.append(short_questions[i[0]])
            sorted_answers.append(answers_int[i[0]])
            short_answers1.append(short_answers[i[0]])

for i in range(3):
    print(sorted_questions[i])
    print(sorted_answers[i])
    print(short_questions1[i])
    print(short_answers1[i])
    print()

[219, 13]
[219, 13, 58, 2310, 3636, 1384, 3365, 220, 2332, 219, 13, 243, 55, 394, 166, 219, 13, 243, 55, 72, 76, 135, 1140, 18574, 5, 4624, 26, 1651, 573, 175, 21, 2363, 193, 76, 4126, 312, 811, 105, 18575, 7875, 175, 1514, 193, 18576, 135, 21, 2363, 26, 21, 3636, 219, 13, 21, 2363, 219, 171]
why can
why can a simple question but yet so complex why can someone do this or why can someone do that i have often pondered for hours to come up with the answer and i believe after years of thoughtprovoking consultation with friends and relativesi have the answer to the question why can the answer why not

[133, 479, 56]
[242, 4123, 3646, 282, 306, 56, 2627, 3924, 201, 918, 5047, 4443, 5, 176, 56, 5048, 193, 21, 2346, 105, 10, 176, 763, 1166, 176, 56, 201, 1916, 763, 193, 1651, 299, 21, 4273, 105, 5049, 5050, 1384, 3685, 84, 6, 29, 202, 10, 3602, 237, 2458, 84, 2379, 299, 21, 2346, 105, 10, 288, 5048, 56, 201, 1168, 20, 21, 2346, 105, 10, 32, 669, 153]
who governs annuities
if youre asking about

In [43]:
def model_inputs():
    input_data = tf.placeholder(tf.int32, [None, None], name="input")
    targets = tf.placeholder(tf.int32, [None, None], name="targets")
    lr = tf.placeholder(tf.float32, name="learning_rate")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    return input_data, targets, lr, keep_prob

def process_encoding_input(target_data, vocab_to_int, batch_size):

    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    enc_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    _, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = enc_cell, cell_bw = enc_cell, sequence_length = sequence_length, inputs = rnn_inputs, dtype=tf.float32)
    return enc_state

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob, batch_size):

    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])

    att_keys, att_vals, att_score_fn, att_construct_fn = tf.keras.seq2seq.prepare_attention(attention_states, attention_option="bahdanau", num_units=dec_cell.output_size)

    train_decoder_fn = tf.keras.seq2seq.attention_decoder_fn_train(encoder_state[0], att_keys, att_vals,  att_score_fn, att_construct_fn,  name = "attn_dec_train")

    train_pred, _, _ = tf.keras.seq2seq.dynamic_rnn_decoder(dec_cell, train_decoder_fn,  dec_embed_input, sequence_length, scope=decoding_scope)
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)

    return output_fn(train_pred_drop)

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob, batch_size):

    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])

    att_keys, att_vals, att_score_fn, att_construct_fn = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option="bahdanau", num_units=dec_cell.output_size)

    train_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0], att_keys, att_vals,  att_score_fn, att_construct_fn,  name = "attn_dec_train")

    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, train_decoder_fn,  dec_embed_input, sequence_length, scope=decoding_scope)
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)

    return output_fn(train_pred_drop)

def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob, batch_size):

    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])

    att_keys, att_vals, att_score_fn, att_construct_fn = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option="bahdanau", num_units=dec_cell.output_size)

    infer_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_inference(output_fn, encoder_state[0],  att_keys, att_vals,  att_score_fn, att_construct_fn,
                        dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size, name = "attn_dec_inf")

    infer_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, infer_decoder_fn, scope=decoding_scope)

    return infer_logits

def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, vocab_to_int, keep_prob, batch_size):

    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        dec_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)

        weights = tf.truncated_normal_initializer(stddev=0.1)
        biases = tf.zeros_initializer()
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, vocab_size, None,  scope=decoding_scope, weights_initializer = weights, biases_initializer = biases)

        train_logits = decoding_layer_train(encoder_state, dec_cell,  dec_embed_input, sequence_length,  decoding_scope, output_fn, keep_prob, batch_size)

        decoding_scope.reuse_variables()
        infer_logits = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, vocab_to_int['<GO>'], vocab_to_int['<EOS>'],
                    sequence_length - 1, vocab_size,  decoding_scope, output_fn, keep_prob, batch_size)

    return train_logits, infer_logits

In [50]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, answers_vocab_size,
                  questions_vocab_size, enc_embedding_size, dec_embedding_size, rnn_size, num_layers,
                  questions_vocab_to_int):

    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, answers_vocab_size+1,  enc_embedding_size, initializer = tf.random_uniform_initializer(0,1))

    enc_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob, sequence_length)

    dec_input = process_encoding_input(target_data, questions_vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.random_uniform([questions_vocab_size+1, dec_embedding_size], 0, 1))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    train_logits, infer_logits = decoding_layer(dec_embed_input, dec_embeddings, enc_state, questions_vocab_size,
                            sequence_length, rnn_size, num_layers, questions_vocab_to_int,  keep_prob, batch_size)

    return train_logits, infer_logits

In [51]:
epochs = 50
batch_size = 64
rnn_size = 512
num_layers = 2
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.005
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.75

### need to modify th code
https://stackoverflow.com/questions/41333798/attributeerror-module-tensorflow-has-no-attribute-interactivesession
https://stackoverflow.com/questions/40782271/attributeerror-module-tensorflow-has-no-attribute-reset-default-graph

In [52]:
tf.reset_default_graph()
# Starting the session
sess = tf.InteractiveSession()

In [53]:
# ops.reset_default_graph()
# # Starting the session
# sess = tf.compat.v1.InteractiveSession()

In [54]:
input_data, targets, lr, keep_prob = model_inputs()
sequence_length = tf.placeholder_with_default(max_line_length, None, name="sequence_length")

In [55]:
input_shape = tf.shape(input_data)

train_logits, inference_logits = seq2seq_model( tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(answers_vocab_to_int),
    len(questions_vocab_to_int), encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers,  questions_vocab_to_int)

AttributeError: module 'tensorflow.contrib.seq2seq' has no attribute 'prepare_attention'

In [106]:
print(tf.random_uniform_initializer(0,1))

In [3]:
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
        
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)


#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial", background="#dddddd")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=130, y=401, height=40, width=265)
SendButton.place(x=6, y=401, height=40)

In [4]:
base.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-3-eb280a6039dd>", line 13, in send
    res = chatbot_response(msg)
NameError: name 'chatbot_response' is not defined
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-3-eb280a6039dd>", line 13, in send
    res = chatbot_response(msg)
NameError: name 'chatbot_response' is not defined
